##Type of Games
1. Perfect Infomation: 双方知道对方的行动
2. Zero-Sum: 只存在一个赢家
3. Deterministic: 没有运气成分

##AI Brute Force Apporach
穷举所有可能的情况，构建一个game tree。这个tree一共有：$9! = 362880$ leaf node。
$$\text{Total Number of Node} = 1! + 2! + ... + 9! = 986409 $$

由于有如此多的可能性，我们需要一种算法去寻找best strategy

##MiniMax
Minimax is a **recursive algorithm** which is used to **choose an optimal move** for a player assuming that the other player is also playing optimally.

在MinMax算法中，一方叫做maximizer, 另一方叫做minimizer. **The maximizer尝试去maximize the scores while the minimizer尝试去minimize the score of maximizer.**

It is called minimax because it helps in minimizing that the other players can force us to receive.

**算法实现：**
1. 构建一个game tree
2. 对每一个leaf node都打一个分 (i.e. win: +1, draw: 0, lose: -1)
3. **evaluate each node**: 如果node属于maxmizer(是一个max node), 那么evaluate的value就是自己所有child node的最大值；如果node属于minimizer(是一个min node), 那么evaluate的value就是自己所有child node的最小值
4. 不断重复步骤3，知道所有node都被evaluate
5. **search:** 对于一个node N, 如果N是一个max node, 走向有最大value的child node；如果N是一个min node, 走向有最小value的child node

##Disadvantage of Minimax
由于Minimax会产生许多的Node, game tree会变得很大。这样一来search和evaluation就会变得很慢。所以我们需要一种方法来减少node数量。这个方法便是：Alpha-Beta Pruning

##Alpha-Beta Pruning
例：

$$layer1: 7 \\
layer2: 7 \quad \quad ? \quad \quad ? \\
layer3: 9 \quad 8 \quad 7 \quad 6 \quad 5 \quad 4 \quad 3 \quad 2 \quad 1$$

我们发现：layer2中第二个node，它的值一定小于6，故我们无需evaluate它的其余child node；同样的，对于第三个node，它的值同样小于3，故我们无需evaluate它的其余child node。

**算法实现：**
1. 定义$\alpha$和$\beta$：$\alpha$ is the **highest** found so far along the path for **MAX Node**; $\beta$ is the **lowest** found so far along the path for **MIN Node**. (当$\alpha \ge \beta$时，剪掉这一支。)
2. 同Minimax一样，构建game tree。但是这次要将$(\alpha, \beta)$作为参数传给child node
3. Update the $(α,β)$ values of all the nodes that have so far been expanded according to:
 $$\text{For MAX node}, α ← max(α,\text{current node value}).$$
 $$\text{For MIN node}, β ← min(β,\text{current node value}).$$
4. Prune all children of any node whose $α ≥ β$.

##Problem:
我们有可能我们的tree过于巨大，我们的recursion depth可能不够到达leaf node, 我们应该如何解决这个问题？

##Evaluation function
使用Evaluation function:

**Estimate** desirability or utility of position. **Correlate with the actual chance of winning**

**Requirements:**
1. Computation of evaluation function values is **efficient**.
2. Evaluation function **agrees with utility function on terminal states**.
3. Evaluation function **accurately reflects the chances of winning**.

大多数game-playing programs使用一个weighted linear function:
$$\omega_1f_1 + \omega_2f_2 + ... + \omega_nf_n$$
来作为Evaluation function。在这里，我么可以使用AI来学习这里weight和feature。

**Learning good evaluation functions automatically from past experience is a promising new direction.**

**Example:**

对Tic-Tac_Toe，我们可以设计一个evaluation function: $e(p)$
1. If p is not a winning position for either player, then $e(p)$ = (the number of
 complete rows, columns, or diagonals that are still open for MAX)- (the
 number of complete rows, columns, or diagonals that are still open for MIN).
2. If p is a win for MAX, then $e(p) = ∞$
3. if p is a win for MIN, then $e(p) = −∞$.

In [ ]:
# MiniMax Code
def drawboard(board):
    print("Current state: \n\n")
    for i in range (0, 9):
        if board[i] == 0:             # 0: empty
            print("- ",end = " ")
        if board[i] == 1:             # 1: X
            print("X ",end = " ")
        if board[i] ==-1:             # -1: O
            print("O ",end = " ")
        if (i+1) % 3 == 0:
            print("\n")
    print("\n")

In [ ]:
def check_game(board):
    winning_pos = [ [0,1,2], [3,4,5], [6,7,8], [0,3,6], [1,4,7], [2,5,8], [0,4,8], [2,4,6] ]
    # Check all possible winning patterns for each player.
    for i in range(0,8):
        if board[winning_pos[i][0]] != 0 and\
        board[winning_pos[i][0]] == board[winning_pos[i][1]] and\
        board[winning_pos[i][0]] == board[winning_pos[i][2]]:
            return board[winning_pos[i][2]]
    if any(element == 0 for element in board): return 2        # game not finish
    # game draw:
    return 0

In [ ]:
def human_turn(board):
    pos = input("Enter O's position [1 to 9]: ")
    pos = int(pos)
    if board[pos - 1] != 0:
      print("Illegal Move!")      # this block is occupied
      exit(0)
    board[pos - 1] = -1

In [ ]:
def minimax(board, player):
    global count
    count += 1
    result = check_game(board)    # Check if any player wins
    if result != 2:
      return result               # 1: AI win;  -1: otherwise

    # 模拟下棋
    scores = []
    for i in range(0,9):
      if board[i] == 0:
        board[i] = player
        scores.append(minimax(board, player * -1))
        board[i] = 0              # undo the trial

    # return max score if player is AI; otherwise return the min score
    return max(scores) if player == 1 else min(scores)

In [ ]:
def ai_turn(board):
    pos =-1
    max_value =-2 # Initialize max value so far to -2,

    for i in range(0,9):
        if board[i] == 0:
          board[i] = 1
          score = minimax(board,-1) # step 4: Calculate minimax score for the human player
          board[i] = 0              # Undo the trial
          if score > max_value:
            max_value = score
            pos = i                 # step 5: search the best position
    # put X, where is the best move
    board[pos] = 1

In [ ]:
 # Initalize the board to all zeros (i.e., all empty cells)
board = [ 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
print("Computer: X VS. You: O")
first = input("Play first (Y/N) :")

if first == 'Y' or first == 'y': # If first is 'Y'/'y', human wants to play first
    play_first = 1
else:
    play_first = 0
for i in range (0,9):
    if check_game(board) != 2:
        break
    if (i+play_first) % 2 == 0: # Take turns to play the game
        count = 0
        ai_turn(board)
        print("Count:", count)
    else:
        drawboard(board)
        human_turn(board)

result = check_game(board)
if result==0:
    drawboard(board)
    print("Draw!!!")
if result==1:
    drawboard(board)
    print("AI(X) Wins! Human(O) Loses!")
if result==-1:
    drawboard(board)
    print("Human(O) Wins! AI(X) Loses!")